In [33]:
import simpy
import numpy as np
import random
import re

In [121]:
RANDOM_SEED = 112
NEW_CUSTOMERS = 500  # Total number of customers
CAPACITY = 51
wait_times = np.zeros(NEW_CUSTOMERS)

def source(env, number, interval, counter, time_needed):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, f'Customer{i:2d}', counter, time_in_bank=time_needed)
        env.process(c)
        t = random.expovariate(1 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    with counter.request() as req:
        # Wait for the counter
        yield req
        start_time = env.now
#         print(f'start time:{start_time}')
#         print(f'arrive : {arrive}')
        wait = env.now - arrive

        # We got to the counter

        tib = random.expovariate(1.0 / time_in_bank)
        yield env.timeout(tib)
#         print(f'time busy: {env.now - start_time}')
    number = re.findall("\d+", name)[0]
#     print(f'wait time: {wait}')
    wait_times[int(number)] = wait
    
# Setup and start the simulation
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run for N=1
N = 1 # Amount of queues
ARRIVAL_RATE = 50/N  # Generate new customers roughly every x seconds
counter = simpy.Resource(env, capacity=N)
env.process(source(env, NEW_CUSTOMERS, ARRIVAL_RATE, counter, CAPACITY))
env.run()
load = N * (ARRIVAL_RATE / CAPACITY)
print(f"Amount of counters: {N}")
print(f"Load on machines: {load}")
print(f'Average wait time per person: {np.sum(wait_times)/NEW_CUSTOMERS}')


# Setup and start the simulation
random.seed(RANDOM_SEED)
env = simpy.Environment()
# Start processes and run for N=2
N = 2 # Amount of queues
ARRIVAL_RATE = 50/N  # Generate new customers roughly every x seconds
counter = simpy.Resource(env, capacity=N)
env.process(source(env, NEW_CUSTOMERS, ARRIVAL_RATE, counter, CAPACITY))
env.run()
load = N * (ARRIVAL_RATE / CAPACITY)
print(f"Amount of counters: {N}")
print(f"Load on machines: {load}")
print(f'Average wait time per person: {np.sum(wait_times)/NEW_CUSTOMERS}')


# Setup and start the simulation
random.seed(RANDOM_SEED)
env = simpy.Environment()
# Start processes and run for N=4
N = 4 # Amount of queues
ARRIVAL_RATE = 50/N  # Generate new customers roughly every x seconds
counter = simpy.Resource(env, capacity=N)
env.process(source(env, NEW_CUSTOMERS, ARRIVAL_RATE, counter, CAPACITY))
env.run()
load = N * (ARRIVAL_RATE / CAPACITY)
print(f"Amount of counters: {N}")
print(f"Load on machines: {load}")
print(f'Average wait time per person: {np.sum(wait_times)/NEW_CUSTOMERS}')

Amount of counters: 1
Load on machines: 0.9803921568627451
Average wait time per person: 687.036467268177
Amount of counters: 2
Load on machines: 0.9803921568627451
Average wait time per person: 584.9277877755459
Amount of counters: 4
Load on machines: 0.9803921568627451
Average wait time per person: 126.91645645905506
